# Imports

In [ ]:
from IPython.display import clear_output

In [ ]:
!pip install -q bertopic

In [ ]:
!pip install -U numpy==1.23.5

In [ ]:
import pandas as pd
from ast import literal_eval
import numpy as np
import os
import random
import re

from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sentence_transformers import SentenceTransformer
from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired, MaximalMarginalRelevance
from bertopic.vectorizers import ClassTfidfTransformer




**Before we start, here is a summary derived from the excellent EDA done in this notebook :**https://www.kaggle.com/code/abaojiang/lmsys-detailed-eda

**General findings**

- There are 64 different models in the training set.
- For each prompt, there are 3 responses, each from different model and ranked according to the human preference.
- The most common (frequent models) in the training data are as follows :

    - gpt-4-1106-preview 
    - gpt-3.5-turbo-0613
    - gpt-4-0613 
    - claude-2.1 
    - (gpt-4-0314,claude-instant-1)
    
    
- Number of Turns : The number of prompt/response pairs in the training data.

    - Around 86.88% of conversations are single-turn.
    - Over 99.19% of conversations are less than 6 turns.
    - The maximum number of turns is 36.
    
    

**Model response preferences**

- Three LLMs have win rate over 50%, including gpt-3.5-turbo-0314, gpt-4-0125-preview and gpt-4-1106-preview.
- A lower tie rate means that a winner can be judged more deterministically


**Biases and correlations in response/prompts**


- No position bias for human judges. i.e , the positions A,B,C in the training data have no such positional bias in them in context of preference of annotators.
 
- Correlation between response lenghts of models for the same prompt: There exist a strong correlation between response length for the same prompt.
 
- Correlation between prompt length and response length : the linear relationship between prompt length and response length, the correlation seems much weaker.

- Verbosity Bias (how the verbosity of the answer affects human preference) : 
    - There is a clear verbosity bias with the data.
    - Correlation of "Mean response length" vs "win rate"  =  0.488
    - gpt-4-0125-preview and gpt-4-1106-preview are models with the top-2 longest average response length.
    

**null/empty responses or prompts**

- There exists 5 samples with empty prompts.
- All of the empty prompts are a single space " " and appear at the last prompt during conversation.
- Models can still continue to respond even if an empty prompt is sent.
- Missing responses can be empty or None.

- efffect on judges
    - We can see that the tie rate drops to around 0.15, which is quite reasonble.
    - If only one model has missing responses, judges might tend to vote the other responding normally or tie.
    

**checking train and test data**

In [ ]:
def parse_response(response_object:str)->[str,]:
    try:
        resp = literal_eval(response_object)
        
    except Exception as e:
        try:
            response_object = response_object.replace('null',None)
            resp = literal_eval(response_object)
        except Exception as e:
#             print(type(response_object),response_object)
            resp = []

    return resp

In [ ]:
train = pd.read_csv("/kaggle/input/lmsys-chatbot-arena/train.csv")
train['prompt'] = train['prompt'].apply(literal_eval)
train['response_a'] = train['response_a'].apply(parse_response)
train['response_b'] = train['response_b'].apply(parse_response)

test = pd.read_csv("/kaggle/input/lmsys-chatbot-arena/test.csv")

_ = print(train.shape),print(test.shape)

In [ ]:
train.head()

In [ ]:
#dropping empty responses
train_ss = train[(train.response_a.apply(len)>0) & (train.response_b.apply(len)>0)]

train_ss.shape

In [ ]:
test.head()

# Topic Modelling 

**How it works (from: https://maartengr.github.io/BERTopic/algorithm/algorithm.html)**

![image.png](attachment:60ac0df8-5866-4948-9240-7e6453976a10.png)

1. Embed documents : We start by converting our documents to numerical representations using a tranformer based model embedding.

2. Dimensionality reduction : After having created our numerical representations of the documents we have to reduce the dimensionality of these representations. Cluster models typically have difficulty handling high dimensional data due to the curse of dimensionality.

3. Cluster Documents : After having reduced our embeddings, we can start clustering our data. For that, we leverage a density-based clustering technique, HDBSCAN. It can find clusters of different shapes and has the nice feature of identifying outliers where possible.

4. Bag-of-words : To create topic representations in BERTopic's algorithm while allowing for modularity, HDBSCAN is used as a clustering model because it accommodates clusters with varying densities and shapes. Instead of using a centroid-based method, all documents in a cluster are combined into a single document, and the frequency of each word is counted to form a bag-of-words representation. This representation, normalized for cluster size differences, focuses on words at the cluster level without assuming a specific cluster structure.

5. Topic representation : From the generated bag-of-words representation, we want to know what makes one cluster different from another. Which words are typical for cluster 1 and not so much for all other clusters? To solve this, we need to modify TF-IDF such that it considers topics (i.e., clusters) instead of documents.

6. Fine-tune Topic representation : we can consider the c-TF-IDF generated topics to be candidate topics. They each contain a set of keywords and representative documents that we can use to further fine-tune the topic representations. Having a set of representative documents for each topic is huge advantage as it allows for fine-tuning on a reduced number of documents. This reduces computation for large models as they only need to operate on that small set of representative documents for each topic. 



    Let us try to find the underlying patterns in the prompt, using a unsupervised topic modelling approach. This will give an high level idea of what the constituent topics in the prompts are.

In [ ]:
# Setup modules


# Step 1 - Extract embeddings
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# Step 2 - Reduce dimensionality
umap_model = UMAP(
    n_neighbors=20, 
    n_components=5,
    min_dist=0.0, 
    metric="cosine",
    random_state=7,
)

# Step 3 - Cluster reduced embeddings
hdbscan_model = HDBSCAN(
    min_cluster_size=32, 
    min_samples=1,
    metric="euclidean", 
    cluster_selection_method="eom",
    prediction_data=True
)

# Step 4 - Tokenize topics
vectorizer_model = CountVectorizer(stop_words="english")

# Step 5 - Create topic representation
ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True,)

# Fine-tune topic representations with  a `bertopic.representation` model
representation_model = MaximalMarginalRelevance(diversity=0.4,
                                                top_n_words=15
                                               )

In [ ]:
# Build topic modeling pipeline
topic_model = BERTopic(
    embedding_model=embedding_model,
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    vectorizer_model=vectorizer_model,
    ctfidf_model=ctfidf_model,
    representation_model=representation_model,
    n_gram_range=(1,5),
    language="english"
)

In [ ]:
train_prompt_concatenated = train.prompt.apply(lambda x: "\n\n".join(x)).to_list()
len(train_prompt_concatenated)

In [ ]:
%%time
topics,topic_proba = topic_model.fit_transform(train_prompt_concatenated)

In [ ]:
topic_model.save("topic_model_unguided", serialization="safetensors")


In [ ]:
print(f" number of unique topics: {len(np.unique(topics))}")


In [ ]:
topic_info = topic_model.get_topic_info()
topic_info.head(10)

In [ ]:
#get a specific topic repr 
topic_model.get_topic(0)

In [ ]:
#visualize topic repr
topic_model.visualize_barchart(top_n_topics=30,n_words=10)

In [ ]:
topic_model.visualize_term_rank()


In [ ]:
topic_model.visualize_heatmap(top_n_topics=20)

# Guided topic modelling 

    The following illustration gives a idea behind guided topic modelling (from: https://maartengr.github.io/BERTopic/getting_started/guided)

![image.png](attachment:e350c127-4025-4b0a-9942-3768abb34873.png)

In [ ]:
task_topics = "Code to text,,text to code,Named entity recognition,Sentiment Analysis,Translation,Question Answering,Program Execution, Miscallenous tasks,Text Categorization,Language Identification, Information Extraction,Text Quality,Summarization,text completion,essay writing,poem writing,creative writing,fact verification,reasoning,mathematical,grammer task,rephrasing,style transfer,paraphrasing,natural language inference,question generation,text matching,dialogue generation,harmfullness detection,toxic language detection,fact verification,keyword tagging".split(",")

print(task_topics)

**Modified task keywords using CHATGPT**

In [ ]:
#define seed topics 
task_topics_modified = [
    ['Code to text', 'source code', 'comments', 'explanation', 'description', 'documentation'],
    ['Text to code', 'programming', 'syntax', 'function', 'script', 'automation'],
    ['Named entity recognition', 'NER', 'entities', 'classification', 'annotation', 'identification'],
    ['Sentiment Analysis', 'emotion', 'opinion', 'polarity', 'attitude', 'mood'],
    ['Translation', 'bilingual', 'language pair', 'conversion', 'interpretation', 'localization'],
    ['Question Answering', 'QA', 'response', 'inquiry', 'knowledge', 'retrieval'],
    ['Program Execution', 'run', 'execute', 'compile', 'script', 'process'],
    ['Miscellaneous tasks', 'varied', 'general', 'diverse', 'assorted', 'multiple'],
    ['Text Categorization', 'classification', 'labeling', 'sorting', 'grouping', 'organization'],
    ['Language Identification', 'detection', 'recognition', 'classification', 'language', 'dialect'],
    ['Information Extraction', 'data mining', 'retrieval', 'extraction', 'parsing', 'harvesting'],
    ['Text Quality', 'clarity', 'readability', 'coherence', 'accuracy', 'precision'],
    ['Summarization', 'abstract', 'condense', 'overview', 'digest', 'outline'],
    ['Text completion', 'autocomplete', 'fill-in', 'predictive', 'continuation', 'suggestion'],
    ['Essay writing', 'composition', 'argument', 'thesis', 'structure', 'drafting'],
    ['Poem writing', 'verse', 'rhyme', 'stanza', 'meter', 'lyric'],
    ['Creative writing', 'story', 'imagination', 'narrative', 'fiction', 'expression'],
    ['Fact verification', 'truth', 'validation', 'accuracy', 'confirmation', 'authenticity'],
    ['Reasoning', 'logic', 'deduction', 'inference', 'rationale', 'analysis'],
    ['Mathematical', 'calculation', 'formula', 'equation', 'computation', 'arithmetic'],
    ['Grammar task', 'syntax', 'rules', 'correction', 'structure', 'editing'],
    ['Rephrasing', 'paraphrase', 'reword', 'rewrite', 'restatement', 'alteration'],
    ['Style transfer', 'transformation', 'conversion', 'adaptation', 'modification', 'recasting'],
    ['Paraphrasing', 'rewording', 'restating', 'rephrasing', 'altering', 'modifying'],
    ['Natural language inference', 'NLI', 'hypothesis', 'entailment', 'contradiction', 'inference'],
    ['Question generation', 'inquiry', 'query', 'interrogative', 'ask', 'question'],
    ['Text matching', 'similarity', 'comparison', 'alignment', 'correlation', 'matching'],
    ['Dialogue generation', 'conversation', 'interaction', 'exchange', 'communication', 'chatbot'],
    ['Harmfulness detection', 'toxicity', 'abuse', 'malice', 'danger', 'risk'],
    ['Toxic language detection', 'abusive', 'offensive', 'harmful', 'inappropriate', 'insulting'],
    ['Fact verification', 'validation', 'authenticity', 'accuracy', 'truth', 'confirmation'],
    ['Keyword tagging', 'labeling', 'annotation', 'classification', 'indexing', 'tagging'],
    ['Topic modeling', 'themes', 'topics', 'clustering', 'segmentation', 'grouping'],
    ['Contextual embedding', 'context', 'representation', 'vectors', 'embeddings', 'contextualization'],
    ['Coreference resolution', 'pronouns', 'anaphora', 'antecedents', 'referents', 'binding'],
    ['Semantic similarity', 'meaning', 'relation', 'comparison', 'equivalence', 'likeness'],
    ['Document summarization', 'overview', 'digest', 'abstract', 'compendium', 'condensation'],
    ['Speech recognition', 'transcription', 'audio', 'voice', 'ASR', 'spoken'],
    ['Optical character recognition', 'OCR', 'text', 'image', 'scanning', 'extraction'],
    ['Text generation', 'creation', 'synthesis', 'generation', 'writing', 'production'],
    ['Dialogue summarization', 'conversation', 'overview', 'recap', 'condensation', 'summary'],
    ['Data anonymization', 'privacy', 'masking', 'obfuscation', 'anonymity', 'de-identification']
]



len(task_topics_modified)

In [ ]:
topic_model_guided = BERTopic(
    embedding_model=embedding_model,
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    vectorizer_model=vectorizer_model,
    ctfidf_model=ctfidf_model,
    representation_model=representation_model,
    n_gram_range=(1,5),
    language="english",
    seed_topic_list=task_topics_modified)

In [ ]:
%%time
topics,topic_proba = topic_model_guided.fit_transform(train_prompt_concatenated)

In [ ]:
topic_model_guided.visualize_barchart(top_n_topics=30,n_words=10)

In [ ]:
topic_model_guided.save("topic_model_guided", serialization="safetensors")


# Resources 



* https://research.google/pubs/large-language-models-are-effective-text-rankers-with-pairwise-ranking-prompting/
* https://magazine.sebastianraschka.com/p/tips-for-llm-pretraining-and-evaluating-rms
* https://magazine.sebastianraschka.com/p/llm-training-rlhf-and-its-alternatives
* https://www.kaggle.com/code/abaojiang/lmsys-detailed-eda
* https://www.kaggle.com/code/robikscube/lmsys-chatbot-arena-data-anaylsis#Response-Length-Baseline
* https://medium.com/data-reply-it-datatech/bertopic-topic-modeling-as-you-have-never-seen-it-before-abb48bbab2b2